In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
rng = np.random.default_rng()

In [16]:
def k_cap(input, cap_size):
        """

        Given a vector input it returns the highest cap_size 
        entries from cap_zie
        """
        output = np.zeros_like(input)
        if len(input.shape) == 1:
            idx = np.argsort(input)[-cap_size:]
            output[idx] = 1
        else:
            idx = np.argsort(input, axis=-1)[:, -cap_size:]
            np.put_along_axis(output, idx, 1, axis=-1)
        return output

In [28]:
class assembly_network: 
    """
    This class is meant to implement the assembly calculus structure
    This generalizes for multiple inputs and brain regions
    
    """

    def __init__(self, number_of_inputs: int , sparsity:int, layers: list, beta: float) -> None:

        """
        Initializes the structure of the Brain Region. It takes the number of inputs and then a list for layers that should contain tuples of the form (neurons, cap_size).

        
        """
        self.n_in = number_of_inputs  # Vector of 28X28 pixels 
         # List with pairs of tuples (n, c) where n is the number of neurons and c is the size of the cap 
        self.create_layers(layers) # Creates all the structure for the brain regions 
        self.sparsity = sparsity
        self.beta =beta  
    

    def create_layers(self, layers)-> None: 
        """ 
        Creates brain regions according to the list from layers 

        The layers list should contain tuples of the form (number of neurons, cap size)
        
        """
        
        self.layers=[]
        temp=self.n_in+0
        for k, (neurons, cap_size) in enumerate(layers):
            self.layers.append(brain_region(neurons, temp, cap_size, k))
            temp=neurons+0
    
        
    def next(self, input: np.array, initial=False, final=False ):
        """
        During the training process, it puts the input 
        through the network and it runs it through all the layers 




        """
    
        temp=input
        print(self.layers)
        for k , brain_region_k in enumerate(self.layers): 
            new_temp=brain_region_k.next(temp, initial=initial, final=final)
            temp=new_temp
            
        return temp 

    def classify(self,input, initial=False ):
        temp=input
        for brain_region in self.layers: 
            print("temp shape"+str(temp.shape))
            temp=brain_region.classify(temp, initial)
        return temp 

        





    


In [27]:
class brain_region: 
    """
    Creates a brain region from assembly calculus 
    
    """

    def __init__(self, n_neurons , n_in, cap_size, id: int ) -> None:
        """
        Creates a brain region that takes 
        """
        self.id=id
        self.n_neurons=n_neurons
        self._n_in=n_in
        self.cap_size=cap_size
        mask = np.zeros((self.n_neurons, self.n_neurons), dtype=bool) # NXN array of zeros 
        W = np.zeros((self.n_neurons, self.n_neurons)) 
        mask_a = np.zeros((self._n_in, self.n_neurons), dtype=bool) # image to N matrix 
        A = np.zeros((self._n_in, self.n_neurons))
        mask = (rng.random((self.n_neurons, self.n_neurons)) < sparsity) & np.logical_not(np.eye(n_neurons, dtype=bool)) # Creating matrix from N to B  with no entries in the diagonal 
        W = np.ones((self.n_neurons, self.n_neurons)) * mask 
        W /= W.sum(axis=0) # Transition probabiliy matrix 
        mask_a = rng.random((self._n_in, self.n_neurons)) < sparsity 
        A = np.ones((self._n_in, self.n_neurons)) * mask_a
        A /= A.sum(axis=0)
        W = np.ones_like(W) * mask
        A = np.ones_like(A) * mask_a
        W /= W.sum(axis=0, keepdims=True)
        A /= A.sum(axis=0, keepdims=True)
        self._W=W
        self._A=A
        self.mask=mask
        self.mask_a=mask_a
        self.act_h = np.zeros(self.n_neurons)
        self.bias = np.zeros(self.n_neurons)
        self.b = -1
          
        self.classify_act_h=np.zeros(self.n_neurons)


    def next(self, input, initial=False, final=False ):
        """
        It computes the activation output of the input going 
        through this brain region. 
        
        """
        if initial:
            self.act_h = np.zeros(self.n_neurons)        

        act_h_new = k_cap(self.act_h @ self._W + input @ self._A + self.bias, self.cap_size)   # output a NXN array for the neurons that are activated. The first part is from self activiation and second from inoput
        self._A[(input > 0)[:, np.newaxis] & (act_h_new > 0)[np.newaxis, :]] *= 1 + beta
        self._W[(self.act_h > 0)[:, np.newaxis] & (act_h_new > 0)[np.newaxis, :]] *= 1 + beta
        self.act_h = act_h_new
        if final: 
            self.reinforce_bias()
        print("Shape of act_h:"+str(self.act_h.shape))
        return self.act_h

    def reinforce_bias(self):
        """
        This function is meant to be called at the end of each round to renormalize the transition matrices 
        
        """
        self.bias[self.act_h>0]+=-1 # after all rounds the activated neurons have a smaller bias so they more likely to fire 
        self._A /= self._A.sum(axis=0, keepdims=True)
        self._W /= self._W.sum(axis=0, keepdims=True)



    def classify(self,input  , initial=False, ):
        if initial:
            self.classify_act_h=np.zeros((input.shape[0] , self.n_neurons))
        
        self.classify_act_h=k_cap(self.classify_act_h @ self._W + input @ self._A + self.bias, self.cap_size)
        return self.classify_act_h

        

In [29]:

class classification_mnist:

    def __init__(self, kernels: list ,train_path: str, test_path: str, number_of_inputs: int , sparsity:int, layers: list, beta: float  ) :
        """

        Creates a MNIST recognition architecture based on assembly calculus
        """
        
        self.cap_size=layers[-1][1]
        self.n_neurons= layers[-1][0]
        self.n_examples=n_examples
        self.n_in=number_of_inputs
        self.assembly_network=assembly_network(number_of_inputs, sparsity, layers, beta , )
        self.get_files( train_path, test_path)
        self.create_training_data(kernels)
        self.create_testing_data(kernels)
        

    def create_training_data(self ,kernels= [np.ones((1, 3, 3))] ):
        """

        Creates a data set with n_examples from the files obtained 
        by get_files 

        """
        self.train_examples = [] 
        for kernel in kernels:
            self.train_examples.append(np.zeros((10, self.n_examples, 784)))
            for i in range(10):
            #Does the convulution between a all 1's 3X3 kernel and each of the images 
                self.train_examples[-1][i] = k_cap(convolve(self.train_imgs[self.train_labels == i][:self.n_examples].reshape(-1, 28, 28), kernel, mode='same').reshape(-1, 28 * 28), self.cap_size)
    
    def create_testing_data(self ,kernels= [np.ones((1, 3, 3))] ):
        """

        Creates a data set with n_examples from the files obtained 
        by get_files 

        """
        self.test_examples = [] 
        for kernel in kernels:
            self.test_examples.append(  np.zeros((10, self.n_examples, 784) ))
            for i in range(10):
            #Does the convulution between a all 1's 3X3 kernel and each of the images 
                self.test_examples[-1][i] = k_cap(convolve(self.test_imgs[self.test_labels == i][:self.n_examples].reshape(-1, 28, 28), kernel, mode='same').reshape(-1, 28 * 28), self.cap_size)
    
    


    def get_files(self, train_path: str, test_path: str)-> None:
        """
        Given two paths it retrieves the data structure encoded in those paths. traun_path should be the path of the training data 
        and test_path should be the path for test data. 
        Assumes a csv format on nthe data on the paths 
        """
        test_data = np.loadtxt(test_path, delimiter=',')

        train_data = np.loadtxt(train_path,  delimiter=',')
        self.train_imgs = train_data[:, 1:]
        self.train_imgs.shape
        self.test_imgs = test_data[:, 1:]
        self.train_labels = train_data[:, 0]
        self.test_labels = test_data[:, 0]

    def train_model(self, n_rounds)-> np.array:
        """
        Given the number of rounds (images that will be shown to the model)

        The program runs and trains the edge weights for the network.
        """
        self.activations = np.zeros((10, n_rounds, self.n_neurons))
        for i in range(10): # iterations for each of the labels 
            for j in range(n_rounds):  # for each of the rounds 
                input = self.train_examples[0][i, j] # image inputs
                act_h= self.assembly_network.next(input, initial=(j==0), final= (j==n_rounds-1) ) # output a NXN array for the neurons that are activated. The first part is from self activiation and second from inoput
                self.activations[i, j] = act_h
        return self.activations

    def classify(self, n_rounds,  test=True )-> dict:
        """
        When called, this function runs one batch of data through 
        the whole network and then returns a dictionary with succes rates

        """
        if test:
            examples=self.test_examples[0]
            
        else:
            examples=self.train_examples[0]
        self.n_examples=examples.shape[1]
        #### RUNS THROUGH NETWORK    
                    
        outputs = np.zeros((10, n_rounds+1, self.n_examples, self.n_neurons))
        for i in np.arange(10):
            for j in range(n_rounds):
                outputs[i, j+1] = self.assembly_network.classify(examples[i], initial= (j==0)) # run each one network for n_rounds and save the neurons active at each step
        #### STARTS CLASSIFICATION 
        c = np.zeros((10, self.n_neurons))
        for i in range(10):
            c[i, outputs[i, 1].sum(axis=0).argsort()[-self.cap_size:]] = 1

        predictions = (outputs[:, 1] @ c.T).argmax(axis=-1)
        acc = (predictions == np.arange(10)[:, np.newaxis]).sum(axis=-1) /examples.shape[1]
        return acc

CLASSIFICATION WITH TWO CONSECUTIVE BRAIN REGIONS

In [30]:
n_in = 784  # Vector of 28X28 pixels 
cap_size = 200 # Size of the cap 
sparsity = 0.1
n_rounds = 10
n_examples=800
beta = 1e0
train_path="./data/mnist/mnist_train.csv"
layers=[ (2000,200)]# number of neurons in network  with respective cap_size
test_path="./data/mnist/mnist_test.csv"
kernels=[np.ones((1, 3, 3))]
classify_two=classification_mnist(kernels,train_path,test_path, n_in , sparsity, layers, beta)
classify_two.train_model( 5)
classify_two.classify( 5, test=False)

Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of ac

array([0.9275 , 0.96   , 0.66375, 0.78125, 0.72875, 0.445  , 0.87625,
       0.7725 , 0.575  , 0.66375])

In [ ]:
array([0.885  , 0.97375, 0.56   , 0.63625, 0.7975 , 0.2325 , 0.71625,
       0.715  , 0.1725 , 0.11875])
       
array([0.9275 , 0.96   , 0.66375, 0.78125, 0.72875, 0.445  , 0.87625,
       0.7725 , 0.575  , 0.66375])

array([0.81875, 0.99   , 0.565  , 0.72375, 0.73875, 0.40625, 0.54   ,
       0.73125, 0.1575 , 0.46875])

In [31]:
n_in = 784  # Vector of 28X28 pixels 
cap_size = 200 # Size of the cap 
sparsity = 0.1
n_rounds = 10
n_examples=800
beta = 1e0
train_path="./data/mnist/mnist_train.csv"
layers=[(2000,400), (2000,200)]# number of neurons in network  with respective cap_size
test_path="./data/mnist/mnist_test.csv"
kernels=[np.ones((1, 3, 3))]
classify_two=classification_mnist(kernels,train_path,test_path, n_in , sparsity, layers, beta)
classify_two.train_model( 20)
classify_two.classify( 20)

[<__main__.brain_region object at 0x00000267259A4040>, <__main__.brain_region object at 0x00000266F6F17580>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x00000267259A4040>, <__main__.brain_region object at 0x00000266F6F17580>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x00000267259A4040>, <__main__.brain_region object at 0x00000266F6F17580>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x00000267259A4040>, <__main__.brain_region object at 0x00000266F6F17580>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x00000267259A4040>, <__main__.brain_region object at 0x00000266F6F17580>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x00000267259A4040>, <__main__.brain_region object at 0x00000266F6F17580>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x00000267259A4040>, <__main__.brain

array([0.885  , 0.97375, 0.56   , 0.63625, 0.7975 , 0.2325 , 0.71625,
       0.715  , 0.1725 , 0.11875])

In [32]:
n_in = 784  # Vector of 28X28 pixels 
cap_size = 200 # Size of the cap 
sparsity = 0.1
n_rounds = 10
n_examples=800
beta = 1e0
train_path="./data/mnist/mnist_train.csv"
layers=[(2000,200), (2000,200)]# number of neurons in network  with respective cap_size
test_path="./data/mnist/mnist_test.csv"
kernels=[np.ones((1, 3, 3))]
classify_two=classification_mnist(kernels,train_path,test_path, n_in , sparsity, layers, beta)
classify_two.train_model( 20)
classify_two.classify( 20)

[<__main__.brain_region object at 0x0000026705340EE0>, <__main__.brain_region object at 0x000002678982FFD0>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026705340EE0>, <__main__.brain_region object at 0x000002678982FFD0>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026705340EE0>, <__main__.brain_region object at 0x000002678982FFD0>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026705340EE0>, <__main__.brain_region object at 0x000002678982FFD0>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026705340EE0>, <__main__.brain_region object at 0x000002678982FFD0>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026705340EE0>, <__main__.brain_region object at 0x000002678982FFD0>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026705340EE0>, <__main__.brain

array([0.81875, 0.99   , 0.565  , 0.72375, 0.73875, 0.40625, 0.54   ,
       0.73125, 0.1575 , 0.46875])

In [33]:
n_in = 784  # Vector of 28X28 pixels 
cap_size = 200 # Size of the cap 
sparsity = 0.1
n_rounds = 10
n_examples=800
beta = 1e0
train_path="./data/mnist/mnist_train.csv"
layers=[(2000,200), (2000,200),(2000,200)]# number of neurons in network  with respective cap_size
test_path="./data/mnist/mnist_test.csv"
kernels=[np.ones((1, 3, 3))]
classify_two=classification_mnist(kernels,train_path,test_path, n_in , sparsity, layers, beta)
classify_two.train_model( 20)
classify_two.classify( 20)

[<__main__.brain_region object at 0x0000026680CAE950>, <__main__.brain_region object at 0x00000266F6E58880>, <__main__.brain_region object at 0x00000266F9CDED10>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026680CAE950>, <__main__.brain_region object at 0x00000266F6E58880>, <__main__.brain_region object at 0x00000266F9CDED10>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026680CAE950>, <__main__.brain_region object at 0x00000266F6E58880>, <__main__.brain_region object at 0x00000266F9CDED10>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026680CAE950>, <__main__.brain_region object at 0x00000266F6E58880>, <__main__.brain_region object at 0x00000266F9CDED10>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026680CAE950>, <__main__.brain_r

array([0.81   , 0.98875, 0.55375, 0.75625, 0.7425 , 0.3825 , 0.535  ,
       0.73125, 0.     , 0.51625])

In [34]:
n_in = 784  # Vector of 28X28 pixels 
cap_size = 200 # Size of the cap 
sparsity = 0.1
n_rounds = 10
n_examples=800
beta = 1e0
train_path="./data/mnist/mnist_train.csv"
layers=[(2000,200), (2000,200),(2000,200),(2000,200)]# number of neurons in network  with respective cap_size
test_path="./data/mnist/mnist_test.csv"
kernels=[np.ones((1, 3, 3))]
classify_two=classification_mnist(kernels,train_path,test_path, n_in , sparsity, layers, beta)
classify_two.train_model( 20)
classify_two.classify( 20)

[<__main__.brain_region object at 0x0000026708D915A0>, <__main__.brain_region object at 0x0000026680CC50C0>, <__main__.brain_region object at 0x00000266E746ABC0>, <__main__.brain_region object at 0x00000266E746AC20>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026708D915A0>, <__main__.brain_region object at 0x0000026680CC50C0>, <__main__.brain_region object at 0x00000266E746ABC0>, <__main__.brain_region object at 0x00000266E746AC20>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026708D915A0>, <__main__.brain_region object at 0x0000026680CC50C0>, <__main__.brain_region object at 0x00000266E746ABC0>, <__main__.brain_region object at 0x00000266E746AC20>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
[<__main__.brain_region object at 0x0000026708D915A0>, <__main__.brain_re

array([0.78375, 0.9925 , 0.55875, 0.74625, 0.73125, 0.38875, 0.5225 ,
       0.71875, 0.     , 0.4775 ])

In [35]:
n_in = 784  # Vector of 28X28 pixels 
cap_size = 200 # Size of the cap 
sparsity = 0.1
n_rounds = 10
n_examples=800
beta = 1e0
train_path="./data/mnist/mnist_train.csv"
layers=[(2000,200), (2000,200),(2000,200),(1000,100)]# number of neurons in network  with respective cap_size
test_path="./data/mnist/mnist_test.csv"
kernels=[np.ones((1, 3, 3))]
classify_two=classification_mnist(kernels,train_path,test_path, n_in , sparsity, layers, beta)
classify_two.train_model( 20)
classify_two.classify( 20)

[<__main__.brain_region object at 0x00000267053420E0>, <__main__.brain_region object at 0x0000026705342E00>, <__main__.brain_region object at 0x000002672D74FF10>, <__main__.brain_region object at 0x000002672D74FCA0>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(1000,)
[<__main__.brain_region object at 0x00000267053420E0>, <__main__.brain_region object at 0x0000026705342E00>, <__main__.brain_region object at 0x000002672D74FF10>, <__main__.brain_region object at 0x000002672D74FCA0>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(1000,)
[<__main__.brain_region object at 0x00000267053420E0>, <__main__.brain_region object at 0x0000026705342E00>, <__main__.brain_region object at 0x000002672D74FF10>, <__main__.brain_region object at 0x000002672D74FCA0>]
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(2000,)
Shape of act_h:(1000,)
[<__main__.brain_region object at 0x00000267053420E0>, <__main__.brain_re

In [ ]:
n_in = 784  # Vector of 28X28 pixels 
cap_size = 200 # Size of the cap 
sparsity = 0.1
n_rounds = 10
n_examples=800
beta = 1e0
train_path="./data/mnist/mnist_train.csv"
layers=[(2000,200), (2000,200)]# number of neurons in network  with respective cap_size
test_path="./data/mnist/mnist_test.csv"
kernels=[np.ones((1, 3, 3))]
classify_two=classification_mnist(kernels,train_path,test_path, n_in , sparsity, layers, beta)
classify_two.train_model( 5)
classify_two.classify( 5)

In [ ]:
n_in = 784  # Vector of 28X28 pixels 
cap_size = 200 # Size of the cap 
sparsity = 0.1
n_rounds = 10
n_examples=800
beta = 1e0
train_path="./data/mnist/mnist_train.csv"
layers=[(2000,200), (1000,100)]# number of neurons in network  with respective cap_size
test_path="./data/mnist/mnist_test.csv"
kernels=[np.ones((1, 3, 3))]
classify_two=classification_mnist(kernels,train_path,test_path, n_in , sparsity, layers, beta)
classify_two.train_model( 5)
classify_two.classify( 5)

In [324]:
n_in = 784  # Vector of 28X28 pixels 
cap_size = 200 # Size of the cap 
sparsity = 0.1
n_rounds = 10
n_examples=800
beta = 1e0
train_path="./data/mnist/mnist_train.csv"
layers=[(2000,200)]# number of neurons in network  with respective cap_size
test_path="./data/mnist/mnist_test.csv"
kernels=[np.ones((1, 3, 3))]
classify_two=classification_mnist(kernels,train_path,test_path, n_in , sparsity, layers, beta)
classify_two.train_model( 20)
classify_two.classify( 20)

ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
ID:0
A shape (784, 2000)
W shape (2000, 200

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [310]:
n_in = 784  # Vector of 28X28 pixels 
cap_size = 200 # Size of the cap 
sparsity = 0.1
n_rounds = 10
n_examples=800
beta = 1e0
train_path="./data/mnist/mnist_train.csv"
layers=[(2000,200)]# number of neurons in network  with respective cap_size
test_path="./data/mnist/mnist_test.csv"
kernels=[np.ones((1, 3, 3))]
classify_two=classification_mnist(kernels,train_path,test_path, n_in , sparsity, layers, beta)
classify_two.train_model( 20)
classify_two.classify( 20)

layer0
input:784
Neurons:2000
Shape of input:(784,)
[(2000, 200)]
layer 0
0
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
(2000,)
Shape of input:(784,)
[(2000, 200)]
layer 0
0
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
(2000,)
Shape of input:(784,)
[(2000, 200)]
layer 0
0
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
(2000,)
Shape of input:(784,)
[(2000, 200)]
layer 0
0
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
(2000,)
Shape of input:(784,)
[(2000, 200)]
layer 0
0
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
(2000,)
Shape of input:(784,)
[(2000, 200)]
layer 0
0
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
(2000,)
Shape of input:(784,)
[(2000, 200)]
layer 0
0
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [305]:

classify_two=classification_mnist(kernels,train_path,test_path, n_in , sparsity, layers, beta)

layer0
input:784
Neurons:2000
layer1
input:2000
Neurons:2000


In [308]:
classify_two.train_model( 20)

Shape of input:(784,)
[(2000, 200), (2000, 200)]
[<__main__.brain_region object at 0x000001BE1C22FE80>, <__main__.brain_region object at 0x000001BE1C22ED40>]
layer 0
0
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
(2000,)
layer 1
1
ID:1
A shape (2000, 2000)
W shape (2000, 2000)
input shape(2000,)
Shape of act_h:(2000,)
(2000,)
Shape of input:(784,)
[(2000, 200), (2000, 200)]
[<__main__.brain_region object at 0x000001BE1C22FE80>, <__main__.brain_region object at 0x000001BE1C22ED40>]
layer 0
0
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
(2000,)
layer 1
1
ID:1
A shape (2000, 2000)
W shape (2000, 2000)
input shape(2000,)
Shape of act_h:(2000,)
(2000,)
Shape of input:(784,)
[(2000, 200), (2000, 200)]
[<__main__.brain_region object at 0x000001BE1C22FE80>, <__main__.brain_region object at 0x000001BE1C22ED40>]
layer 0
0
ID:0
A shape (784, 2000)
W shape (2000, 2000)
input shape(784,)
Shape of act_h:(2000,)
(2000

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan

In [307]:
classify_two.classify( 20)

10
(10, 800, 784)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800, 2000)
temp shape(800, 784)
temp shape(800,

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
#Note -- place your own MNIST files in the appropriate directory

test_data = np.loadtxt("./data/mnist/mnist_test.csv", delimiter=',')
train_data = np.loadtxt("./data/mnist/mnist_train.csv",  delimiter=',')



In [ ]:
# Every row of the CSV file must be an image. The first entry is the label of that image and the rest is the image.

train_imgs = train_data[:, 1:]
test_imgs = test_data[:, 1:]
train_labels = train_data[:, 0]
test_labels = test_data[:, 0]

In [ ]:
#Definitoin of parameters
n_in = 784  # Vector of 28X28 pixels 
n_neurons = 2000 # number of neurons in network 
cap_size = 200 # Size of the cap 
sparsity = 0.1
n_rounds = 5
beta = 1e0


In [ ]:
mask = np.zeros((n_neurons, n_neurons), dtype=bool) # NXN array of zeros 
W = np.zeros((n_neurons, n_neurons)) 
mask_a = np.zeros((n_in, n_neurons), dtype=bool) # image to N matrix 
A = np.zeros((n_in, n_neurons))
mask = (rng.random((n_neurons, n_neurons)) < sparsity) & np.logical_not(np.eye(n_neurons, dtype=bool)) # Creating matrix from N to B  with no entries in the diagonal 
W = np.ones((n_neurons, n_neurons)) * mask 
W /= W.sum(axis=0) # Transition probabiliy matrix 
mask_a = rng.random((n_in, n_neurons)) < sparsity 
A = np.ones((n_in, n_neurons)) * mask_a
A /= A.sum(axis=0)

In [ ]:
print(mask)
print(mask[0])
print(mask.shape)
print(W)
print(W.shape)
print(np.sum(W,axis=0))
print(A)
print(A.shape)
print(np.sum(A, axis=0))

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False  True False]
 [False False False ...  True False False]
 [False False False ... False False False]]
[False False False ... False False False]
(2000, 2000)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.00471698 0.        ]
 [0.         0.         0.         ... 0.00492611 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(2000, 2000)
[1. 1. 1. ... 1. 1. 1.]
[[0.         0.         0.         ... 0.         0.         0.01388889]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.  

In [ ]:
from scipy.signal import convolve

### 
n_examples = 5000
examples = np.zeros((10, n_examples, 784))
for i in range(10):
    #Does the convulution between a all 1's 3X3 kernel and each of the images 
    examples[i] = k_cap( convolve(train_imgs[train_labels == i][:n_examples].reshape(-1, 28, 28), np.ones((1, 3, 3)), mode='same').reshape(-1, 28 * 28), cap_size)

In [ ]:
print(np.ones_like(W).shape)

(2000, 2000)


In [ ]:
W = np.ones_like(W) * mask
A = np.ones_like(A) * mask_a
W /= W.sum(axis=0, keepdims=True)
A /= A.sum(axis=0, keepdims=True)


In [ ]:
print(np.sum(W[W>0], axis=0) )

1999.9999999999995


In [ ]:
bias = np.zeros(n_neurons)
b = -1
activations = np.zeros((10, n_rounds, n_neurons))

In [ ]:
# Training with the number of rounds where the weights are updated at each step 
#
for i in range(10): # iterations for each of the labels 
    act_h = np.zeros(n_neurons) # the plasticity of the neurons
    for j in range(n_rounds):  # for each of the rounds 
        input = examples[i, j] # image inputs 
        act_h_new = k_cap(act_h @ W + input @ A + bias, cap_size)   # output a NXN array for the neurons that are activated. The first part is from self activiation and second from inoput
        activations[i, j] = act_h_new.copy() 
        A[(input > 0)[:, np.newaxis] & (act_h_new > 0)[np.newaxis, :]] *= 1 + beta
        W[(act_h > 0)[:, np.newaxis] & (act_h_new > 0)[np.newaxis, :]] *= 1 + beta
        act_h = act_h_new
    bias[act_h > 0] += b # after all rounds the activated neurons have a smaller bias so they more likely to fire 
    A /= A.sum(axis=0, keepdims=True)
    W /= W.sum(axis=0, keepdims=True)

In [ ]:
### Testing on the training data set 

outputs = np.zeros((10, n_rounds+1, n_examples, n_neurons))
for i in np.arange(10):
    for j in range(n_rounds):
        outputs[i, j+1] = k_cap(outputs[i, j] @ W + examples[i] @ A, cap_size) # run each one network for n_rounds and save the neurons active at each step

In [ ]:
print(outputs.shape)
outputs[0]

(10, 6, 5000, 2000)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]],

    

In [ ]:
idx = np.full(n_neurons, -1, dtype=int) ## array with each of the neurons set to -1 
# act = outputs[:, -1].sum(axis=1)
act = activations[:, -1].copy()  # the last state of the neurons for each of the labels 
for i, j in enumerate(range(10)): ## for each of the labels 
    idx[i*cap_size:(i+1)*cap_size] = act[j].argsort()[-cap_size:][::-1] # put the activation values for the top cap_size neurons in the ith par
    #partition of idx 
    act[:, idx[i*cap_size:(i+1)*cap_size]] = -1 # set the neurons picked already to -1 in each of the layers 
    #This gives priority to lower numbers 
r = np.arange(n_neurons) # creates array wwith number from 0 to n_neurons

# idx[idx>-1] gets the entries 
temp=idx 
r[idx[idx > -1]] = -1 
idx[(i+1)*cap_size:] = np.unique(r)[1:]

In [ ]:
print(temp, temp.shape)

[ 485 1693  282 ... 1045 1988 1989] (2000,)


In [ ]:
r_example=np.arange(n_neurons)
print(r_example, r_example.shape)

[   0    1    2 ... 1997 1998 1999] (2000,)


In [ ]:
print(idx[cap_size*10:])

[]


In [ ]:
fig, axes = plt.subplots(10, n_rounds, figsize=(10, 2 * 10), sharex=True, sharey=True)
for ax, output in zip(axes, outputs):
    for i in range(n_rounds):
        ax[i].imshow((output[i+1] > 0)[:n_neurons, idx])
        ax[i].set_axis_off()
fig.text(0.5, 0.04, 'Neurons', ha='center', va='center')
fig.text(0.04, 0.5, 'Samples', ha='center', va='center', rotation='vertical')
fig.show()

<IPython.core.display.Javascript object>

In [ ]:
fig

<IPython.core.display.Javascript object>

In [ ]:
def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1, keepdims=True)

In [ ]:
v = 0.1 * rng.standard_normal((10, n_neurons))
targets = np.zeros((100, 10))
for i in range(10):
    targets[i*10:(i+1)*10, i] = 1
update = np.zeros_like(v)

In [ ]:
for _ in range(100):
    permutation = rng.permutation(n_examples - 1000)
    for j in range((n_examples - 1000) // 10):
        batch = outputs[:, 1, permutation[j*10:(j+1)*10]].reshape(10 * 10, n_neurons)
        scores = softmax((batch[:, :, np.newaxis] * v.T[np.newaxis, :, :]).sum(axis=1))
        update = 0.5 * update + 1e-3 * (batch[:, np.newaxis, :] * (scores - targets)[:, :, np.newaxis]).sum(axis=0)
        v -= update
#             v = v.clip(min=0.0, max=1.0)
#             v[v < 1e-5] = 0.

KeyboardInterrupt: 

In [ ]:
((outputs[:, 1, :-1000] @ v.T).argmax(axis=-1) == np.arange(10)[:, np.newaxis]).sum() / 40000

0.998375

In [ ]:
((outputs[:, 1, -1000:] @ v.T).argmax(axis=-1) == np.arange(10)[:, np.newaxis]).sum() / 10000

0.9259

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
for i in range(10):
    ax.bar(np.arange(n_neurons), outputs[i, -1].mean(axis=0)[idx], label=i)
ax.legend(loc='upper right', ncol=2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_ylim([0, 1.1])
ax.set_xticklabels([])
ax.set_xlabel('Neurons')
ax.set_ylabel('Firing Probability')

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Firing Probability')

In [ ]:
c = np.zeros((10, n_neurons))
for i in range(10):
    c[i, outputs[i, 1].sum(axis=0).argsort()[-cap_size:]] = 1
#     c[i, idx[i*cap_size:(i+1)*cap_size]] = 1
predictions = (outputs[:, 1] @ c.T).argmax(axis=-1)
acc = (predictions == np.arange(10)[:, np.newaxis]).sum(axis=-1) / n_examples

In [ ]:
acc

array([0.948 , 0.9502, 0.6392, 0.78  , 0.6814, 0.462 , 0.8062, 0.7636,
       0.6048, 0.6792])

In [ ]:
fig, axes = plt.subplots(1, 10, figsize=(10, 2))
for i in range(10):
    axes[i].imshow((A * c[i][np.newaxis, :]).sum(axis=1).reshape(28, 28))
    axes[i].set_axis_off()
fig.tight_layout()

<IPython.core.display.Javascript object>